# Imports

In [1]:
import numpy as np
import scipy.spatial.distance
from ui_script import show_top_5
import pickle
import random
import re

# Load word2vec

In [2]:
from gensim.models.word2vec import Word2Vec
w2v = Word2Vec.load_word2vec_format('../../data/GoogleNews-vectors-negative300.bin', binary=True)
w2v.init_sims(replace=True)

# Load image vectors

In [4]:
new_vec_dict = np.load('../../data/image_space_20170213-1541.npy').item()

# Evaluate random query image

In [15]:
# Random select query image and caption
query_file_name = random.choice(list(new_vec_dict.keys()))
query_vec = new_vec_dict[query_file_name]
query_caption = ' '#w2v.most_similar(query_vec, restrict_vocab=30000, topn=1)[0][0]

# top 5 images
top5_file_names = 5*['']
top5_captions = 5*['']
top5_distances = 5*[float('inf')]
top5_w2v_distances = 5*[float('inf')]

# Iterate over all images
for file_name in new_vec_dict.keys():
    if file_name != query_file_name:

        # See if current image is in top 5
        for i, dist in enumerate(top5_distances):
            temp_dist = scipy.spatial.distance.cosine(query_vec,new_vec_dict[file_name])
            if temp_dist < dist:
                top5_distances[i] = temp_dist
                top5_file_names[i] = '../../data/val2014/' + file_name
                break

## Iterate over all captions
#w2v_dict = np.load('../../data/word2vec_val.npy')
#w2v_dict = w2v_dict[()]
#with open('../../data/caption_val.pkl','rb') as data_file:
#    caption_dict = pickle.load(data_file)
#for file_name in w2v_dict.keys():
#    if file_name != query_file_name:
#
#        # See if current image is in top 5
#       temp_dist = scipy.spatial.distance.cosine(query_vec,w2v_dict[file_name][0])
#        for i, dist in enumerate(top5_w2v_distances):
#            if temp_dist < dist:
#                top5_w2v_distances[i] = temp_dist
##                top5_file_names[i] = '../../../data/train2014/' + file_name
#                top5_captions[i] = caption_dict[file_name][0]
#                break

# Find most similar word in word2vec
for i in range(5):
    file_name = re.search('COCO.+',top5_file_names[i])[0]
    top5_captions[i] = w2v.most_similar(
        np.mean([query_vec,new_vec_dict[file_name]],axis=0), restrict_vocab=30000, topn=1)[0][0] \
            + '(' + str(top5_distances[i]) + ')'

# Display result
file_names = ['../../data/val2014/' + query_file_name] + top5_file_names
captions = [query_caption] + top5_captions
show_top_5(file_names, captions, height = 250, title='Image space evaluation')

Done
